# Stellar Formation Time at $z=0$

In [8]:
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from scipy.stats import binned_statistic

In [9]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup
from auriga.paths import Paths
from auriga.settings import Settings
from auriga.parser import parse

In [10]:
figure_setup()
settings = Settings()

In [11]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

In [12]:
def read_data(simulation: str) -> pd.DataFrame:
    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=DISC_STD_CIRC,
        disc_min_circ=DISC_MIN_CIRC,
        cold_disc_delta_circ=COLD_DISC_DELTA_CIRC,
        bulge_max_specific_energy=BULGE_MAX_SPECIFIC_ENERGY,
    )
    s.tag_in_situ_stars()
    s.add_stellar_age()

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame()
    df["StellarAge_Gyr"] = s.stellar_age[is_real_star & is_main_obj]
    df["RegionTag"] = s.region_tag[is_real_star & is_main_obj]
    df["IsInSitu"] = s.is_in_situ[is_real_star & is_main_obj].astype("bool")

    return df

In [13]:
simulations = [f"au{i}_or_l4_s127" for i in settings.galaxies]

In [14]:
for simulation in simulations:
    df = read_data(simulation=simulation)
    label = f"Au{parse(simulation)[0]}"

    fig = plt.figure(figsize=(7.4, 2.0))
    gs = fig.add_gridspec(nrows=1, ncols=4, hspace=0.0, wspace=0.0)
    axs = gs.subplots(sharex=True, sharey=True)

    for ax in axs.flat:
        ax.tick_params(which='both', direction="in")
        ax.set_xlim(0, 14)
        ax.set_ylim(0, 0.2)
        ax.set_xticks([0, 2, 4, 6, 8, 10, 12])
        ax.set_xlabel('Stellar Age [Gyr]')
        ax.set_ylabel(r"$f_\star$")
        ax.grid(True, ls='-', lw=0.25, c="gainsboro")
        ax.label_outer()

    for idx, component in enumerate(settings.components):

        n_stars_comp = (df["RegionTag"] == idx).sum()

        hist, bin_edges = np.histogram(
                a=df["StellarAge_Gyr"][df["RegionTag"] == idx],
                bins=28, range=(0, 14))
        bin_centers = bin_edges[1:] - np.diff(bin_edges)[0] / 2
        axs[idx].plot(
            bin_centers, hist / n_stars_comp,
            c=settings.component_colors[component], lw=1)

        hist, _ = np.histogram(
                a=df["StellarAge_Gyr"][
                    (df["RegionTag"] == idx) & (df["IsInSitu"] == 1)],
                bins=28, range=(0, 14))
        axs[idx].plot(
            bin_centers, hist / n_stars_comp,
            c='k', lw=0.75, ls=(0, (3, 1.5, 1, 1.5)), label="In-Situ Stars")

        hist, _ = np.histogram(
                a=df["StellarAge_Gyr"][
                    (df["RegionTag"] == idx) & (df["IsInSitu"] == 0)],
                bins=28, range=(0, 14))
        axs[idx].plot(
            bin_centers, hist / n_stars_comp,
            c='k', lw=0.75, ls=(0, (5, 1.5)), label="Ex-Situ Stars")

        axs[idx].text(x=0.05, y=0.95, size=8.0, ha="left", va="top",
                    s=settings.component_labels[settings.components[idx]],
                    transform=axs[idx].transAxes)

    axs[3].legend(loc="lower left", framealpha=0, fontsize=6,
                  bbox_to_anchor=(0.009, 0.65))

    axs[0].text(
        x=0.95, y=0.95, size=8.0,
        s=r"$\texttt{" + label + "}$",
        ha='right', va='top', transform=axs[0].transAxes)

    fig.savefig(
        f"../images/stellar_formation_time/{simulation}{SUFFIX}.pdf")
    plt.close(fig)